68点いった！
Data AugmentationとFineTuningしよう！
わかりやすくなるようにいらんやつけそう

DataAugmentation 完了

## 犬と猫を分類するモデルを構築

### 1. 画像の読み込み

まずは簡単に画像の読み込みの方法を紹介します。

追加

In [1]:
import sys
import pprint
pprint.pprint(sys.path)

['/Users/takekazukitagishi/Desktop/情報学館授業資料/s２/特論７/univ_tokyo_2020/tensorflow',
 '/Applications/anaconda3/lib/python37.zip',
 '/Applications/anaconda3/lib/python3.7',
 '/Applications/anaconda3/lib/python3.7/lib-dynload',
 '',
 '/Applications/anaconda3/lib/python3.7/site-packages',
 '/Applications/anaconda3/lib/python3.7/site-packages/aeosa',
 '/Applications/anaconda3/lib/python3.7/site-packages/IPython/extensions',
 '/Users/takekazukitagishi/.ipython']


In [2]:
sys.path.insert(1,'/Users/takekazukitagishi/Desktop/情報学館授業資料/s２/特論７/univ_tokyo_2020/tensorflow/venv/lib/python3.7/site-packages')

In [3]:
import sys
import pprint
pprint.pprint(sys.path)

['/Users/takekazukitagishi/Desktop/情報学館授業資料/s２/特論７/univ_tokyo_2020/tensorflow',
 '/Users/takekazukitagishi/Desktop/情報学館授業資料/s２/特論７/univ_tokyo_2020/tensorflow/venv/lib/python3.7/site-packages',
 '/Applications/anaconda3/lib/python37.zip',
 '/Applications/anaconda3/lib/python3.7',
 '/Applications/anaconda3/lib/python3.7/lib-dynload',
 '',
 '/Applications/anaconda3/lib/python3.7/site-packages',
 '/Applications/anaconda3/lib/python3.7/site-packages/aeosa',
 '/Applications/anaconda3/lib/python3.7/site-packages/IPython/extensions',
 '/Users/takekazukitagishi/.ipython']


In [4]:
import onnx

追加終わり

In [6]:
import warnings
warnings.filterwarnings('ignore')

`train` のフォルダに入っている画像を読み込みます。
`glob` を利用すると簡単にファイルを検索できます。

In [7]:
from glob import glob

In [8]:
dog_filepaths = glob('train/dog/*.jpg')
cat_filepaths = glob('train/cat/*.jpg')

In [10]:
from PIL import Image

### 2. クラスラベルの割り振り

画像の読み込み方がわかったため、クラスのラベルを割り振っていきましょう。

In [11]:
from keras.preprocessing.image import load_img,img_to_array,ImageDataGenerator,array_to_img
import os
import numpy as np

In [12]:
# 入力値:x, 目標値: t
x, t = [], []

In [13]:
# ImageDataGeneratorをつくる
# datagenは「45度の範囲でランダムに回転するやつ」など
datagen = ImageDataGenerator(rotation_range=45)
datagen2 = ImageDataGenerator(shear_range=0.85)
datagen3 = ImageDataGenerator(horizontal_flip=0.3)
datagen4 = ImageDataGenerator(vertical_flip=0.3)

In [14]:
# 犬
for filepath in dog_filepaths:
    img = Image.open(filepath)
    img = img.resize((224, 224))#追加
    img = np.array(img)
    x.append(img)
    t.append(np.array(0))  # 犬は 0 とする
    
    ###
    
    #ImageDataGeneratorは4次元データじゃないと読み込まないっぽい
    img = img.reshape((1,) + img.shape)
     #img.shapeすると(1,224,224,3)
    g = datagen.flow(img, batch_size=1)
    for i in range(3):
        batches = g.next()
         #batchesは(1,224,224,3)になっているから、batchesの[0]番目の配列を指定してあげる
        x.append(batches[0])
        t.append(np.array(0))
    #面倒くさいからg2,g3,g4も同時にぶちこんでる
    g2 = datagen2.flow(img, batch_size=1)
    for i in range(3):
        batches = g2.next()
        x.append(batches[0])
        t.append(np.array(0))
    g3 = datagen3.flow(img, batch_size=1)
    for i in range(3):
        batches = g3.next()
        x.append(batches[0])
        t.append(np.array(0))
    g4 = datagen4.flow(img, batch_size=1)
    for i in range(3):
        batches = g4.next()
        x.append(batches[0])
        t.append(np.array(0))

In [15]:
# 猫
for filepath in cat_filepaths:
    img = Image.open(filepath)
    img = img.resize((224, 224))#追加
    img = np.array(img)
    x.append(img)
    t.append(np.array(1))  # 猫は 1 とする
    
    ##
        
     #ImageDataGeneratorは4次元データじゃないと読み込まないっぽい
    img = img.reshape((1,) + img.shape)
     #img.shapeすると(1,224,224,3)
    g = datagen.flow(img, batch_size=1)
    for i in range(3):
        batches = g.next()
         #batchesは(1,224,224,3)になっているから、batchesの[0]番目の配列を指定してあげる
        x.append(batches[0])
        t.append(np.array(1))
    #面倒くさいからg2,g3,g4も同時にぶちこんでる
    g2 = datagen2.flow(img, batch_size=1)
    for i in range(3):
        batches = g2.next()
        x.append(batches[0])
        t.append(np.array(1))
    g3 = datagen3.flow(img, batch_size=1)
    for i in range(3):
        batches = g3.next()
        x.append(batches[0])
        t.append(np.array(1))
    g4 = datagen4.flow(img, batch_size=1)
    for i in range(3):
        batches = g4.next()
        x.append(batches[0])
        t.append(np.array(1))
    

In [16]:
# 格納された枚数
len(x), len(t)

(3900, 3900)

In [17]:
# 全体を numpy の形式に変換
x = np.array(x)  # f は float32
t = np.array(t)  # i は int32

In [18]:
# 300(枚), 224(height), 224(width), 3(channels)
x.shape

(3900, 224, 224, 3)

In [19]:
t.shape

(3900,)

x の値の範囲を正規化しておくと学習が効率的に進むことが多いため、こちらも行なっておきましょう。学習係数などを調整する場合など、必ず必要ではありません。

In [20]:
# 特徴量の正規化 (0~1の範囲に)
x = x / 255

### 3. 訓練データと検証データに分割

tensorflow では訓練データ (train) と検証データ (val) を分ける機能がないため、scikit-learn の `train_test_split` を利用します。

なお、テストデータ (test) は私の手元にしかないため、手持ちのデータの中で以下にテストデータを模擬的に表現できるかのために検証データがあります。精度を高めたいときには**交差検証法 (cross validation)** も選択肢としてあるため、こちらも必要であれば挑戦してみましょう。

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
# train : val = 0.7 : 0.3 の割合で分割する
train_x, val_x, train_t, val_t = train_test_split(x, t, train_size=0.7, random_state=0)

In [23]:
# サイズの確認（訓練データ）
train_x.shape, train_t.shape

((2730, 224, 224, 3), (2730,))

In [24]:
# サイズの確認（検証データ）
val_x.shape, val_t.shape

((1170, 224, 224, 3), (1170,))

### 4. モデルを定義

今回は非常にシンプルな構成の CNN でモデルを構築します。あまり良い精度が出ないようにしてあるため、ここのモデルの構造を工夫して精度を高めましょう。

ヒント
- VGG, ResNet, MobileNet
- ファインチューニング

複雑なモデルを構築する場合には GPU がないと遅い場合もあるため、[こちらの手順](https://www.kikagaku.ai/tutorial/guide_for_beginners/learn/platform_environment) を参考に Google Colab を使ってみると良いでしょう。

In [25]:
import tensorflow as tf

In [26]:
# バージョンの情報
tf.__version__

'2.3.0'

モデル構築の際に乱数のシードも固定しておきましょう。

In [27]:
import os
import random

def reset_seed(seed=0):

    os.environ['PYTHONHASHSEED'] = '0'
    random.seed(seed) #　random関数のシードを固定
    np.random.seed(seed) #numpyのシードを固定
    tf.random.set_seed(seed) #tensorflowのシードを固定

In [28]:
# シードの固定を実行
reset_seed()

tensorflow のモデルの定義に必要なモジュールを読み込みます

In [29]:
from tensorflow.keras import models, layers

以下のようなモデルを定義していきましょう。

- Original: (224, 224, 3)
- => Convolution (Relu) => (224, 224, 6)
- => Half Pooling => (112, 112, 6)
- => ベクトル化 (112×112×6)
- => 全結合層 (112×112×6 => 100), Relu
- => 全結合層 (100 => 2), Softmax

In [31]:
conv_base = models.Sequential()

conv_base.add(layers.Conv2D(3, kernel_size=(3,3), padding='same', activation='relu', input_shape=(224, 224, 3)))
conv_base.add(layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))


In [32]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

ファインチューニング

In [33]:
conv_base.trainable = False

### 5. モデルの学習

Tenrsoflow では定義したモデルをコンパイルする必要がありました。最適化のアルゴリズムを確率的勾配降下法 (SGD) として選択します。

ここのアルゴリズムの選択と学習係数の設定も精度向上のポイントですので、[こちらの記事](https://www.tensorflow.org/guide/keras/train_and_evaluate) などを参考にしながら進めてみましょう。

In [34]:

# モデルのコンパイル
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [35]:
from keras import optimizers

In [36]:
"""
# モデルのコンパイル
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.RMSprop(lr=2e-5),
    metrics=['acc']
)
"""

"\n# モデルのコンパイル\nmodel.compile(\n    loss='binary_crossentropy',\n    optimizer=optimizers.RMSprop(lr=2e-5),\n    metrics=['acc']\n)\n"

In [37]:
# モデルの学習
history = model.fit(
    train_x, train_t,
    batch_size=256,
    #epochs=2,
    epochs=20,
    validation_data=(val_x, val_t)
)

Epoch 1/20
11/11 [==============================] - 9s 775ms/step - loss: 1.2837 - accuracy: 0.5044 - val_loss: 0.8310 - val_accuracy: 0.5000
Epoch 2/20
11/11 [==============================] - 8s 744ms/step - loss: 0.6572 - accuracy: 0.5952 - val_loss: 0.6472 - val_accuracy: 0.5983
Epoch 3/20
11/11 [==============================] - 8s 758ms/step - loss: 0.5598 - accuracy: 0.7168 - val_loss: 0.5598 - val_accuracy: 0.7641
Epoch 4/20
11/11 [==============================] - 8s 756ms/step - loss: 0.5035 - accuracy: 0.8073 - val_loss: 0.5594 - val_accuracy: 0.6778
Epoch 5/20
11/11 [==============================] - 10s 874ms/step - loss: 0.4687 - accuracy: 0.7974 - val_loss: 0.4984 - val_accuracy: 0.7718
Epoch 6/20
11/11 [==============================] - 8s 755ms/step - loss: 0.4233 - accuracy: 0.8473 - val_loss: 0.4732 - val_accuracy: 0.7991
Epoch 7/20
11/11 [==============================] - 9s 799ms/step - loss: 0.3924 - accuracy: 0.8615 - val_loss: 0.4330 - val_accuracy: 0.8376
Epoch

train に関して accuracy が 0.76, validation に関しては accuracy が 0.49 となっており、**過学習 (overfitting)** が生じていることがわかります。

この辺りも踏まえて、予測の精度を上げられるように色々な手法に挑戦してみましょう。比較的簡単なところでは以下を調整してみると良いでしょう。

- モデルの構成（VGG, ResNet, MobileNet など）
- ファイチューニング
- 最適化アルゴリズムの選択
- 学習係数

ファインチューニング

model をconv_baseと全結合層に分けて、conv_baseだけを取り出す

一回、model.add()でも大丈夫かを確認しよう→大丈夫だった

compileを変えて、ファインチューニングのをやってみよう

In [38]:
"""
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      #epochs=30,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2)
      """

"\nmodel.compile(loss='binary_crossentropy',\n              optimizer=optimizers.RMSprop(lr=2e-5),\n              metrics=['acc'])\n\nhistory = model.fit_generator(\n      train_generator,\n      steps_per_epoch=100,\n      #epochs=30,\n      epochs=1,\n      validation_data=validation_generator,\n      validation_steps=50,\n      verbose=2)\n      "

In [39]:
conv_base.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 3)       84        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 3)       0         
Total params: 84
Trainable params: 0
Non-trainable params: 84
_________________________________________________________________


In [40]:

conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'conv2d':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False


In [41]:
"""
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      #epochs=100,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=50)
"""

"\nmodel.compile(loss='binary_crossentropy',\n              optimizer=optimizers.RMSprop(lr=1e-5),\n              metrics=['acc'])\n\nhistory = model.fit_generator(\n      train_generator,\n      steps_per_epoch=100,\n      #epochs=100,\n      epochs=1,\n      validation_data=validation_generator,\n      validation_steps=50)\n"

In [42]:
"""
# モデルのコンパイル
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.RMSprop(lr=1e-5),
    metrics=['acc']
)
"""

"\n# モデルのコンパイル\nmodel.compile(\n    loss='binary_crossentropy',\n    optimizer=optimizers.RMSprop(lr=1e-5),\n    metrics=['acc']\n)\n"

In [43]:
# モデルのコンパイル
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# モデルの学習
history = model.fit(
    train_x, train_t,
    batch_size=256,
    epochs=10,
    #epochs=100,
    #epochs=30,
    validation_data=(val_x, val_t)
)

Epoch 1/10
11/11 [==============================] - 17s 2s/step - loss: 1.0613 - accuracy: 0.7161 - val_loss: 0.5070 - val_accuracy: 0.7248
Epoch 2/10
11/11 [==============================] - 14s 1s/step - loss: 0.3147 - accuracy: 0.8996 - val_loss: 0.3808 - val_accuracy: 0.8590
Epoch 3/10
11/11 [==============================] - 15s 1s/step - loss: 0.2627 - accuracy: 0.9348 - val_loss: 0.3528 - val_accuracy: 0.8291
Epoch 4/10
11/11 [==============================] - 15s 1s/step - loss: 0.1889 - accuracy: 0.9476 - val_loss: 0.2864 - val_accuracy: 0.8735
Epoch 5/10


### 5. モデルの保存

推論にて使用するために学習済みモデルをファイルに出力しましょう。

In [ ]:
# HDF5 という形式で保存（TensorFlowではこちらを用いるようです）
model.save('dog_cat_cnn.h5')

これで作業フォルダの中に `dog_cat_cnn.h5` というファイルが出力できていれば成功です。

### 7. ONNX形式で出力

ONNX は TensorFlow や PyTorch 問わず、モデルの形式を標準化するプロジェクトです。  
最近では、こちらの形式に統一しておくほうが汎用性が高くなっているため、推論サーバーでは ONNX 形式を使用します。

onnxruntime では通常よりも速度が速いと言われています。

In [ ]:
# Windows
# !pip install onnxruntime   or   !python -m pip install onnxruntime

# macOS
#!pip3 install onnxruntime

In [ ]:
import onnxruntime
onnxruntime.__version__

In [ ]:
# Windows
# !pip install keras2onnx  or   !python -m pip install keras2onnx

# macOS
#!pip3 install keras2onnx

In [ ]:
#import onnx
import keras2onnx
keras2onnx.__version__

In [ ]:
# Keras -> ONNX の変換
onnx_model = keras2onnx.convert_keras(model, model.name)

In [ ]:
# モデルを保存
keras2onnx.save_model(onnx_model, 'dog_cat_cnn88.onnx')

In [ ]:
# モデルの読み込み
session = onnxruntime.InferenceSession('dog_cat_cnn.onnx')

In [ ]:
# 推論に必要な構造の抽出
session.get_modelmeta()
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

In [ ]:
input_name, output_name

In [ ]:
# 推論
y_probs = session.run([output_name], {input_name: x})[0]
y_probs

In [ ]:
# ラベル
y = np.argmax(y_prob)
y